# Modeling

In [2]:
# Importing libraries

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2 as pg

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

### Approach 2

In [17]:
# Querying data for approach 2, outlined here: https://docs.google.com/spreadsheets/d/1Y7bXGM6dVQzLsd64f7TeoxQni1sNSX3DraimmwMeEYc/edit#gid=1634158638

query = '''
    SELECT *
    FROM prod.features
'''

conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

df = pd.read_sql_query(query, conn)
df_train = df[(df['Season Type'] == 'Regular') & (df['Season'] < 2018)]
df_dev = df[(df['Season Type'] == 'NCAA Tourney') & (df['DayNum'] < 999) & (df['Season'] < 2018)]
# df_test = df[(df[Season Type'] == 'NCAA Tourney') & (df['DayNum'] < 999) & (df['Season'] == 2018)]

In [39]:
df_train_X = df_train.loc[:, df_train.columns != 'Outcome']
df_dev_X = df_dev.loc[:, df_dev.columns != 'Outcome']

In [40]:
df_train_Y = df_train.loc[:, df_train.columns == 'Outcome']
df_dev_Y = df_dev.loc[:, df_dev.columns == 'Outcome']

In [30]:
df_train.shape

(153272, 40)

In [19]:
df_dev.shape

(1962, 40)

In [21]:
# df_test.shape

(0, 40)

In [22]:
df_train[df_train['Season Type'] == "Regular"].head()

,Season,DayNum,Team,Opponent,Outcome,Score,OpponentScore,NumOT,WLoc,Season Type,...,SeedDiff,OpponentAvgPointsFor,OpponentAvgPointsAgainst,OpponentAvgNetPointsFor,TourWins,OpponentTourWins,TourWinsDiff,cum_wins_v_opp,cum_games_v_opp,cum_winpct_v_opp
0,2003,26,1103,1324,1.0,88.0,84.0,0.0,A,Regular,...,NaN,95.0000,68.0,27.00000,0.0,0.0,0.0,NaN,NaN,NaN
1,2003,77,1115,1108,0.0,50.0,79.0,0.0,H,Regular,...,NaN,66.5882,76.0,-9.41177,0.0,0.0,0.0,NaN,NaN,NaN
2,2003,20,1105,1435,0.0,40.0,68.0,0.0,H,Regular,...,NaN,80.0000,58.0,22.00000,0.0,4.0,-4.0,NaN,NaN,NaN
3,2003,71,1155,1314,0.0,66.0,68.0,0.0,H,Regular,...,NaN,70.7143,65.5,5.21429,5.0,46.0,-41.0,NaN,NaN,NaN
4,2003,30,1156,1103,1.0,68.0,65.0,0.0,A,Regular,...,NaN,81.5000,81.0,0.50000,2.0,0.0,2.0,NaN,NaN,NaN


In [23]:
df_dev[df_dev['Season Type'] == "NCAA Tourney"].head()

,Season,DayNum,Team,Opponent,Outcome,Score,OpponentScore,NumOT,WLoc,Season Type,...,SeedDiff,OpponentAvgPointsFor,OpponentAvgPointsAgainst,OpponentAvgNetPointsFor,TourWins,OpponentTourWins,TourWinsDiff,cum_wins_v_opp,cum_games_v_opp,cum_winpct_v_opp
326,2003,137,1120,1386,1.0,65.0,63.0,1.0,N,NCAA Tourney,...,3.0,70.6207,59.2759,11.34480,10.0,4.0,6.0,0.0,0.0,NaN
334,2003,138,1113,1242,0.0,76.0,108.0,0.0,N,NCAA Tourney,...,8.0,81.3333,66.3667,14.96670,3.0,42.0,-39.0,0.0,0.0,NaN
335,2003,144,1120,1393,0.0,78.0,79.0,0.0,N,NCAA Tourney,...,7.0,80.1034,69.8966,10.20690,10.0,27.0,-17.0,0.0,0.0,NaN
355,2003,137,1122,1257,0.0,64.0,86.0,0.0,N,NCAA Tourney,...,9.0,81.8333,68.4667,13.36670,1.0,21.0,-20.0,0.0,0.0,NaN
855,2003,137,1280,1139,0.0,46.0,47.0,0.0,N,NCAA Tourney,...,-7.0,68.5862,60.9310,7.65517,7.0,1.0,6.0,0.0,0.0,NaN


In [31]:
df_train.head()

,Season,DayNum,Team,Opponent,Outcome,Score,OpponentScore,NumOT,WLoc,Season Type,...,SeedDiff,OpponentAvgPointsFor,OpponentAvgPointsAgainst,OpponentAvgNetPointsFor,TourWins,OpponentTourWins,TourWinsDiff,cum_wins_v_opp,cum_games_v_opp,cum_winpct_v_opp
0,2003,26,1103,1324,1.0,88.0,84.0,0.0,A,Regular,...,NaN,95.0000,68.0,27.00000,0.0,0.0,0.0,NaN,NaN,NaN
1,2003,77,1115,1108,0.0,50.0,79.0,0.0,H,Regular,...,NaN,66.5882,76.0,-9.41177,0.0,0.0,0.0,NaN,NaN,NaN
2,2003,20,1105,1435,0.0,40.0,68.0,0.0,H,Regular,...,NaN,80.0000,58.0,22.00000,0.0,4.0,-4.0,NaN,NaN,NaN
3,2003,71,1155,1314,0.0,66.0,68.0,0.0,H,Regular,...,NaN,70.7143,65.5,5.21429,5.0,46.0,-41.0,NaN,NaN,NaN
4,2003,30,1156,1103,1.0,68.0,65.0,0.0,A,Regular,...,NaN,81.5000,81.0,0.50000,2.0,0.0,2.0,NaN,NaN,NaN


In [41]:
df_train_X.head()

,Season,DayNum,Team,Opponent,Score,OpponentScore,NumOT,WLoc,Season Type,holdout_s1,...,SeedDiff,OpponentAvgPointsFor,OpponentAvgPointsAgainst,OpponentAvgNetPointsFor,TourWins,OpponentTourWins,TourWinsDiff,cum_wins_v_opp,cum_games_v_opp,cum_winpct_v_opp
0,2003,26,1103,1324,88.0,84.0,0.0,A,Regular,NaN,...,NaN,95.0000,68.0,27.00000,0.0,0.0,0.0,NaN,NaN,NaN
1,2003,77,1115,1108,50.0,79.0,0.0,H,Regular,NaN,...,NaN,66.5882,76.0,-9.41177,0.0,0.0,0.0,NaN,NaN,NaN
2,2003,20,1105,1435,40.0,68.0,0.0,H,Regular,NaN,...,NaN,80.0000,58.0,22.00000,0.0,4.0,-4.0,NaN,NaN,NaN
3,2003,71,1155,1314,66.0,68.0,0.0,H,Regular,NaN,...,NaN,70.7143,65.5,5.21429,5.0,46.0,-41.0,NaN,NaN,NaN
4,2003,30,1156,1103,68.0,65.0,0.0,A,Regular,NaN,...,NaN,81.5000,81.0,0.50000,2.0,0.0,2.0,NaN,NaN,NaN


In [42]:
df_train_Y.head()

,Outcome
0,1.0
1,0.0
2,0.0
3,0.0
4,1.0


In [43]:
df_dev_X.head()

,Season,DayNum,Team,Opponent,Score,OpponentScore,NumOT,WLoc,Season Type,holdout_s1,...,SeedDiff,OpponentAvgPointsFor,OpponentAvgPointsAgainst,OpponentAvgNetPointsFor,TourWins,OpponentTourWins,TourWinsDiff,cum_wins_v_opp,cum_games_v_opp,cum_winpct_v_opp
326,2003,137,1120,1386,65.0,63.0,1.0,N,NCAA Tourney,0.0,...,3.0,70.6207,59.2759,11.34480,10.0,4.0,6.0,0.0,0.0,NaN
334,2003,138,1113,1242,76.0,108.0,0.0,N,NCAA Tourney,0.0,...,8.0,81.3333,66.3667,14.96670,3.0,42.0,-39.0,0.0,0.0,NaN
335,2003,144,1120,1393,78.0,79.0,0.0,N,NCAA Tourney,0.0,...,7.0,80.1034,69.8966,10.20690,10.0,27.0,-17.0,0.0,0.0,NaN
355,2003,137,1122,1257,64.0,86.0,0.0,N,NCAA Tourney,0.0,...,9.0,81.8333,68.4667,13.36670,1.0,21.0,-20.0,0.0,0.0,NaN
855,2003,137,1280,1139,46.0,47.0,0.0,N,NCAA Tourney,0.0,...,-7.0,68.5862,60.9310,7.65517,7.0,1.0,6.0,0.0,0.0,NaN


In [44]:
df_dev_Y.head()

,Outcome
326,1.0
334,0.0
335,0.0
355,0.0
855,0.0
